In [4]:
import pandas as pd
import glob

In [20]:
# Checking data filenames
DATA_FILENAMES = glob.glob("data/*.csv")
DATA_FILENAMES

['data/asset_details.csv',
 'data/supplemental_train.csv',
 'data/train.csv',
 'data/example_test.csv']

In [22]:
# Checking that we only have the 4 files
assert len(DATA_FILENAMES) == 4

In [18]:
# Importing and storing asset details
ASSET_DETAILS = pd.read_csv(filepath_or_buffer="data/asset_details.csv", index_col="Asset_ID")
ASSET_DETAILS

,Weight,Asset_Name
Asset_ID,,
2,2.397895,Bitcoin Cash
0,4.304065,Binance Coin
1,6.779922,Bitcoin
5,1.386294,EOS.IO
7,2.079442,Ethereum Classic
6,5.894403,Ethereum
9,2.397895,Litecoin
11,1.609438,Monero
13,1.791759,TRON


In [30]:
TRAIN = pd.read_csv(filepath_or_buffer="data/train.csv")

In [32]:
TRAIN["timestamp"] = pd.to_datetime(arg=TRAIN["timestamp"], 
                                    unit="s", errors="ignore")

In [45]:
TRAIN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  datetime64[ns]
 1   Asset_ID   int64         
 2   Count      float64       
 3   Open       float64       
 4   High       float64       
 5   Low        float64       
 6   Close      float64       
 7   Volume     float64       
 8   VWAP       float64       
 9   Target     float64       
dtypes: datetime64[ns](1), float64(8), int64(1)
memory usage: 1.8 GB


In [53]:
TRAIN["timestamp"][1]

Timestamp('2018-01-01 00:01:00')

In [66]:
TRAIN["Row_ID"] = TRAIN.index

In [65]:
#TRAIN.reset_index(inplace=True)

In [59]:
#TRAIN.set_index(["Row_ID", "timestamp"], inplace=True)

In [64]:
TRAIN.iloc[2,]

level_0         2.000000
index           2.000000
Asset_ID        1.000000
Count         229.000000
Open        13835.194000
High        14013.800000
Low         13666.110000
Close       13850.176000
Volume         31.550062
VWAP        13827.062093
Target         -0.014643
Name: (2, 2018-01-01 00:01:00), dtype: float64

In [52]:
TRAIN[TRAIN["timestamp"] == pd.Timestamp('2018-01-02 02:01:00')]

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
12056,2018-01-02 02:01:00,2,121.0,2388.386667,2399.900,2377.7000,2389.143333,23.545853,2389.098753,0.003012
12057,2018-01-02 02:01:00,0,41.0,8.450200,8.480,8.4501,8.450100,4550.280000,8.453034,0.003290
12058,2018-01-02 02:01:00,1,680.0,13679.416000,13857.900,13616.2400,13696.650000,79.799739,13689.085731,-0.000019
12059,2018-01-02 02:01:00,5,42.0,8.429100,8.437,8.3667,8.370900,3431.836659,8.396779,0.019333
12060,2018-01-02 02:01:00,7,35.0,30.766500,30.993,30.5370,30.767500,3043.400768,30.768720,0.014947
12061,2018-01-02 02:01:00,6,799.0,846.585000,852.760,827.0000,844.477500,1952.468799,843.873541,0.048078
12062,2018-01-02 02:01:00,9,355.0,238.752000,240.880,236.6200,238.624000,1861.348662,238.789950,0.037191
12063,2018-01-02 02:01:00,11,12.0,355.765000,357.350,354.1600,356.160000,15.135289,355.892475,0.014257


In [67]:
# TODO
    # Split le dataframe de train en plusieurs dataframes: 1 par Asset_ID 